# Парсинг HTML, XPath

### выполнил - Колеганов Н.Д.

### Задание

Написать приложение, которое собирает основные новости с сайтов mail.ru, lenta.ru, yandex-новости. Для парсинга использовать XPath. Структура данных должна содержать:

    • название источника;

    • наименование новости;

    • ссылку на новость;

    • дата публикации.


In [0]:
from pprint import pprint
from datetime import datetime
from lxml import html
import requests

## Функции для сбора

In [0]:
    headers = {
                'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/80.0.3987.87 Chrome/80.0.3987.87 Safari/537.36'
            }

с сайта Lenta.ru

In [0]:
def requests_to_lenta():
    main_link='https://lenta.ru/'
    req=requests.get(main_link, headers=header)
    root=html.fromstring(req.text)
    news_list=[]

    #Распарсим главную новость
    title=root.xpath("//div[@class='first-item']//h2/a/text()")
    if title:
        link=root.xpath("//div[@class='first-item']//h2/a/@href")
        news_dict=dict()
        news_dict['site']=main_link
        news_dict['title']=title[0].replace(u'\xa0', u' ')
        news_dict['link']=link[0]
        news_dict['time']=root.xpath("//div[@class='first-item']//h2/a/time/text()")
        news_list.append(news_dict)
    else:
        print('Main news not found on lenta.ru')

    #Распарсим остальные новости
    titles=root.xpath("//section[@class='row b-top7-for-main js-top-seven']"
                      "//div[@class='item']//a/text()")
    if len(titles)>0:
        links=root.xpath("//section[@class='row b-top7-for-main js-top-seven']"
                         "//div[@class='item']//a/@href")
        for i in range(len(links)):
            news_dict=dict()
            news_dict['site']=main_link
            news_dict['title']=titles[i].replace(u'\xa0', u' ')
            news_dict['link']=links[i]
            news_dict['time']=datetime.now().strftime("%d-%m-%Y %H:%M")
            news_list.append(news_dict)
    else:
        print('News not found on lenta.ru')

    #Выведем результат на экран
    pprint(news_list)

с сайта Mail.ru

In [0]:
def requests_to_mail():
    main_link='https://mail.ru/'
    req=requests.get(main_link, headers=header)
    root=html.fromstring(req.text)
    news_list=[]

    #Распарсим главную новость
    title=root.xpath("//div[@class='news-item o-media "
                       "news-item_media news-item_main']//h3/text()")
    if title:
        link=root.xpath("//div[@class='news-item o-media news-item_media "
                          "news-item_main']//a/@href")
        news_dict=dict()
        news_dict['site']=main_link
        news_dict['title']=title[0].replace(u'\xa0', u' ')
        news_dict['link']=link[0]
        news_dict['time']=datetime.now().strftime("%d-%m-%Y %H:%M")
        news_list.append(news_dict)
    else:
        print('Main news not found on mail.ru')

    #Распарсим остальные новости
    titles=root.xpath("//div[@class='news-item__inner']/a[not(contains(@class,"
                        " 'i-color-black i-inline'))]/text()")
    if len(titles)>0:
        links=root.xpath("//div[@class='news-item__inner']/a[not(contains"
                           "(@class, 'i-color-black i-inline'))]/@href")
        for i in range(len(links)):
            news_dict=dict()
            news_dict['site']=main_link
            news_dict['title']=titles[i].replace(u'\xa0', u' ')
            news_dict['link']=links[i]
            news_dict['time']=datetime.now().strftime("%d-%m-%Y %H:%M")
            news_list.append(news_dict)
    else:
        print('News not found on mail.ru')

    #Выведем результат на экран
    pprint(news_list)

In [0]:

def get_news_mail_ru():
    
    news = []    
    
    keys = ('title', 'date', 'link')
    date_format = '%Y-%m-%dT%H:%M:%S%z'
    

    link_mail_ru = 'https://mail.ru/'
    
    request = requests.get(link_mail_ru, headers=headers)
    root = html.fromstring(request.text)
    
    news_links = root.xpath('''(//div[@class =  "news-item o-media news-item_media news-item_main"]  |  
                                //div[@class =  "news-item__inner"])
                                /a[contains(@href, "news.mail.ru")]/@href''')
    
    news_text = root.xpath('''(//div[@class =  "news-item o-media news-item_media news-item_main"]//h3  |  
                               //div[@class =  "news-item__inner"]/a[contains(@href, "news.mail.ru")])
                               /text()''')
    
    for i in range(len(news_text)):
        news_text[i] = news_text[i].replace(u'\xa0', u' ')
    
    news_links_temp = []
    for item in news_links:
        item = item.split('/')
        news_links_temp.append('/'.join(item[0:5]))
        
    news_links = news_links_temp
    del(news_links_temp)
    
    news_date = []
    
    for item in news_links:
        request = requests.get(item, headers=headers)
        root = html.fromstring(request.text)
        date = root.xpath('//span[@class="note__text breadcrumbs__text js-ago"]/@datetime')
        news_date.extend(date)
        
    for i in range(len(news_date)):
        news_date[i] = datetime.strptime(news_date[i], date_format)
        
    for item in list(zip(news_text, news_date, news_links)):
        news_dict = {}
        for key, value in zip(keys, item):
            news_dict[key] = value
        
        news_dict['source'] = 'mail.ru'
        news.append(news_dict)
    
    return news

## Результат

In [26]:
get_news_mail_ru()

[]

In [25]:
requests_to_lenta()

[{'link': '/news/2020/03/17/ukaz/',
  'site': 'https://lenta.ru/',
  'time': ['20:52'],
  'title': 'Путин назначил дату голосования по поправкам к Конституции'},
 {'link': '/news/2020/03/17/333/',
  'site': 'https://lenta.ru/',
  'time': '17-03-2020 19:21',
  'title': 'Евросоюз запретит въезд гражданам других стран'},
 {'link': '/news/2020/03/17/blaise/',
  'site': 'https://lenta.ru/',
  'time': '17-03-2020 19:21',
  'title': 'Чемпион мира по футболу заразился коронавирусом'},
 {'link': '/news/2020/03/17/zharov/',
  'site': 'https://lenta.ru/',
  'time': '17-03-2020 19:21',
  'title': 'Глава Роскомнадзора покинет ведомство'},
 {'link': '/news/2020/03/17/peskov_ukaz/',
  'site': 'https://lenta.ru/',
  'time': '17-03-2020 19:21',
  'title': 'Песков объяснил вероятность переноса голосования за поправки к '
           'Конституции'},
 {'link': '/news/2020/03/17/neffft/',
  'site': 'https://lenta.ru/',
  'time': '17-03-2020 19:21',
  'title': 'Цена на нефть упала ниже 29 долларов'},
 {'link